In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import HeatMap

plt.style.use("seaborn-muted")
sns.set_style('darkgrid')
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 200)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

%matplotlib inline

# Contexto do problema

Esse é um desafio proposto pelo Meigarom do blog [Seja um data scientist](http://sejaumdatascientist.com) e consiste da seguinte situação:

" A **House Rocket** é uma plataforma digital que tem como modelo de negócio a compra e venda de imóveis usando tecnologia. Sua principal estratégia é comprar boas casas em ótimas localizações com preços baixos e depois revendê-las posteriormente à preços mais altos. Quanto maior a diferença entre a compra e a venda, maior o lucro da empresa e portanto maior sua receita. Entretanto, as casas possuem muitos atributos que as tornam mais ou menos atrativas aos compradores e vendedores e a localização e o período do ano também podem influenciar os preços. Portanto a meta é responder as seguinte perguntas:
1. Quais casas o CEO da House Rocket deveria comprar e por qual preço de compra?
1. Uma vez a casa em posse da empresa, qual o melhor momento para vendê-las e qual seria o preço da venda?
1. A House Rocket deveria fazer uma reforma para aumentar o preço da venda? Quais seriam as sugestões de mudanças?"

Minha meta é responder todas essas perguntas através de uma análise dos dados. NÃO irei realizar um tratamento dos dados (padronizar, escalonar, imputer) e nem aplicar um modelo de ML nesse notebook, apenas em um próximo.

# Exploração dos dados

## Carregamento

In [ ]:
raw_data = pd.read_csv('../input/housesalesprediction/kc_house_data.csv', parse_dates=['date', 'yr_built'])

## Visualização prévia

Antes de mais nada, vamos à descrição de cada coluna dos dados
* **date**: data de venda
* **price**: preço de venda
* **bedrooms**: número de quartos
* **bathrooms**: número de banheiros
* **sqft_living**: tamanho da área habitacional em pés²
* **sqft_lot**: tamanho do lote em pés²
* **floors**: número de andares
* **waterfront**: ‘1’ se a propriedade for à beira-mar, ‘0’ se não.
* **view**: um índice de 0 a 4 do quão bom é a visão da propriedade (imagine 0 para uma propriedade com visão para um beco sujo e 4 para uma propriedade com visão para um lindo parque)
* **condition**: condição da casa, com valores de 1 até 5
* **grade**: classificação pela qualidade do material da casa. Construções com melhores materiais normalmente custam mais caro
* **sqft_above**: pés² acima do solo
* **sqft_basement**: pés² abaixo do solo
* **yr_built**: ano de construção
* **yr_renovated**: ano de renovação. ‘0’ se nunca foi renovada
* **zipcode**: código zip de 5 dígitos
* **lat, long**: latitude e longitude
* **squft_livng15**: tamanho médio das 15 casas mais próximas, em pés²
* **sqft_lot15**: tamanho médio dos lotes das 15 casas mais próximas, em pés²

De início, o dataset não possui uma grande quantidade de variáveis, porém todas parecem ser úteis, então não precisarei fazer um PCA ou RFE mais tarde. Verei a influência das mesmas abaixo

In [ ]:
raw_data.head(10)

Logo de cara, podemos ver alguns dados estranhos, como os números 4.5, 2.25 e 1.5 na coluna de banheiros.

In [ ]:
print('Dimensão dos dados')
print('Linhas:',raw_data.shape[0])
print('Colunas:',raw_data.shape[1])

É um dataset com uma quantidade razoável de amostras

In [ ]:
raw_data.info()

In [ ]:
raw_data = raw_data.assign(year_built=raw_data.yr_built.dt.year)
raw_data.drop('yr_built', 1, inplace=True)

In [ ]:
raw_data.head()

Peguei apenas as informações dos anos de construção e coloquei em uma nova coluna, depois excluir a coluna antiga que também continha dia e mês.

## Exploração da variável alvo (price)

In [ ]:
pd.DataFrame(raw_data['price'].describe())

Há um desvio-padrão muito elevado, algo que pode significar a presença de vários outliers. Além disse, é possível notar a grande diferença entre a residência do Q1 (quartil 1) que custa **75.000** e a residência do Q3 (quartil 3) que custa **645.000**, sem falar do valor máximo de **7.700.000**.

In [ ]:
med_price = raw_data['price'].median()

plt.figure(figsize=[20, 8])
sns.distplot(raw_data['price'], color = 'r', label = 'Distribuição')
plt.axvline(med_price, color='b', linestyle='dashed', label='Mediana')
plt.ticklabel_format(style='plain', axis='x')
plt.title('Distribuição do preço de vendas das casas')
plt.xlabel('Preço (em dólar)')
plt.ylabel('Densidade')
plt.xticks(np.arange(raw_data['price'].min(), raw_data['price'].max(), step=500000))

plt.legend()
plt.show()

Está longe de ser uma distribuição normal. Há uma forte assimetria à direita. Grande parte das residências se encontra com preço aproximadamente entre **250.000** ~ **575.000** dólares. À partir de $ 1.075.000 a quantidade de casas tem uma grande diminuição e essa tendência continua até o valor máximo.

## Exploração das demais variáveis

### Valores nulos

In [ ]:
raw_data.isnull().sum().sort_values(ascending=False)

Por se tratar de um dataset do Kaggle, já era esperado que os dados tivessem passado por algum tratamento prévio, hipótese confirmada com o código acima, que mostra que não há variáveis nulas em cada coluna.

### Colunas importantes

In [ ]:
plt.figure(figsize=[12, 6])

plt.subplot(121)
sns.boxplot(x='condition', y='price', data=raw_data);

plt.subplot(122)
sns.boxplot(x='grade', y='price', data=raw_data)

plt.show()

A mediana das casas não variou muito em relação à "condition", porém há muitos outliers, principalmente nos valores 3, 4 e 5. Verificarei melhor abaixo. Já a variável "grade" parece influenciar de maneira positiva o preço das casas. Há diversos outliers, porém a mediana do preço aumenta quanto melhor a qualidade dos materiais.

In [ ]:
pd.DataFrame(raw_data['condition'].value_counts().sort_index())

Há poucas casas em condições ruins (1, 2), uma quantidade elevada de casas em condição média (3), uma quantidade considerável em condições boas (4) e uma quantidade menor de casas em condições ótimas (5).

In [ ]:
pd.DataFrame(raw_data['grade'].value_counts().sort_index())

A variável "grade" parece seguir uma distribuição normal, onde quantidade de casas aumenta de 1 a 7 e após isso começa a diminuir até as casas de "grade" 13.

#### Correlação entre colunas

In [ ]:
var_num = raw_data._get_numeric_data()
var_num.drop('id', 1, inplace=True)
var_num.head()

In [ ]:
var_num.corr()

In [ ]:
var_num_corr = var_num.corr()

plt.figure(figsize = [12, 8])
sns.heatmap(var_num_corr, vmin=-1, vmax=1, linewidth=0.01, linecolor='black', cmap='RdBu_r')
plt.show()

É notável que existem diversas colunas sem correlação com o preço de venda. Abaixo analisarei melhor as colunas que possuem alguma correlação, seja positiva ou negativa.

#### Correlação das colunas com o *price*

In [ ]:
var_num_corr['price'].sort_values(ascending=False).round(3)

Irei analisar melhor as colunas que possuem correlação maior que 0.5 e menor que -0.5, nesse caso **bathrooms, sqft_living, grade, sqft_above e sqft_living15**. Apesar da coluna **bedrooms**  não apresentar uma correlação muito forte, irei usá-la nas análises abaixo, já que no conceito de negócio casas com mais quartos costumam custar mais.

In [ ]:
cols = raw_data[['bedrooms', 'bathrooms', 'sqft_living', 'grade', 'sqft_above',
                         'sqft_living15', 'price']]
most_corr_var = cols.corr()

plt.figure(figsize=[10, 6])
sns.heatmap(data=most_corr_var, vmin=-1, vmax=1, linewidth=0.01, linecolor='black', cmap='RdBu_r', annot=True)

plt.show()

Como dito, as colunas possuem forte correlação positiva, e isso nos diz por exemplo que quanto maior a quantidade de banheiro, maior o preço de venda. Irei analisar melhor essa relação abaixo:

#### Variáveis que podem aumentar o preço das casas

In [ ]:
plt.figure(figsize=[15, 15])

i = 1

for col in cols:
    if col == 'price':
        continue
    plt.subplot(4, 2, i)
    sns.regplot(raw_data[col], raw_data['price'], line_kws={'color': 'r'})
    plt.xlabel(col)
    plt.ylabel('Preço de venda ($)')
    i+=1
    

plt.tight_layout()
plt.show()

Com esses gráficos podemos observar as seguintes informações:
- O valor das residências tende a aumentar de acordo com o número de quartos, porém a partir de 6 quartos esse valor tende a cair, o que pode significar que o número de quartos a partir de 6 não tem tanta influência no preço, o que confirma a informação do matriz que correlação feita anteriormente;
- O número de banheiros tem influência positiva direta no preço das residências, apesar de haver muita dispersão dos dados a partir de 5 banheiros. E de novo é possível notar que esse número de banheiros possui números _float_ , o que é algo estranho. Provavelmente um erro na hora da coleta dos dados;
- **sqft_living** também possui influência direta, apesar da grande dispersão a partir 7000;
- Como já visto, "grade" também tem influência positiva;
- **sqft_above** e **sqft_living15** possuem influência positiva, apesar da ambos possuirem dispersão a partir de 5000;

### Colunas não tão importantes

Agora irei visualizar melhor as colunas que possuem menos correlação com a variável de preço.

#### View

In [ ]:
raw_data['view'].value_counts() / len(raw_data)

A grande maioria das casas de encontra na condição 0, seguido por 2, 3, 1 e 4

In [ ]:
plt.figure(figsize=(8, 4))
sns.set_style("darkgrid")

sns.regplot(raw_data['view'], raw_data['price'], line_kws={'color': 'r'})

plt.show()

Independente da qualidade de "view", as casas costumam ter preço de até 4.000.000, sem contar com os _outliers_.

#### Waterfront

In [ ]:
plt.figure(figsize=(12, 6))

sns.scatterplot(raw_data['waterfront'], raw_data['price'], hue=raw_data['waterfront'])

plt.show()

É possível notar que o valor das casas varia entre o mínimo e o máximo, independente de possuirem "waterfront" (indicado pelo número 1.0) ou não (indicado pelo número 0.0)

#### Floors

In [ ]:
plt.figure(figsize=(12, 6))

cmap = sns.cubehelix_palette(dark=.8, light=.3, as_cmap=True)
sns.scatterplot(raw_data['floors'], raw_data['price'], hue=raw_data['floors'], palette=cmap)

plt.show()

Apesar dos *outliers*, grande parte das residências costumam custar até 4.000.000, independente do andar.

### Separando os anos de construção em décadas

In [ ]:
dates = pd.DataFrame(raw_data['year_built'], columns=['year_built'])
bins = [1900, 1910, 1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010, 2015]
labels = ['1900 - 1910', '1911 - 1920', '1921 - 1930', '1931 - 1940', '1941 - 1950', '1951 - 1960', '1961 - 1970', '1971 - 1980',
         '1981 - 1990', '1991 - 2000', '2001 - 2010', '2011 - 2015']
raw_data['decade_built'] = pd.cut(dates['year_built'], bins, labels = labels, include_lowest = True)
raw_data.sample(5)

In [ ]:
plt.figure(figsize=[12, 8])

sns.barplot(x=raw_data['price'], y=raw_data['decade_built'], palette="cubehelix_d")

plt.show()

Após uma variação de altos e baixos entre os anos de 1900 e 1960, o preço das casas voltou a subir a partir do ano de 1961.

#### Separando os preços das casas em grupos

In [ ]:
prices = pd.DataFrame(raw_data['price'], columns=['price'])
bins = [0, 250000, 500000, 1000000, 8000000]
labels = ['Group 1', 'Group 2', 'Group 3', 'Group 4']
raw_data['price group'] = pd.cut(prices['price'], bins, labels = labels, include_lowest = True)
raw_data.sample(5)

Crei uma nova coluna separando as casas em grupos de acordo com o preço:
* Group 1: de 0 a 250000 dólares
* Group 2: de 250001 a 500000 dólares
* Group 3: de 500001 a 1000000 dólares
* Group 4: acima de 1000001 dólares

In [ ]:
raw_data.groupby('price group')['price group'].count()

Como visto no histograma inicial, a grande maioria das casas vendidas se encontra nos grupos 2 e 3, ou seja, a faixa de preço varia entre 250 mil doláres e 1 milhão.

#### Criando uma nova coluna mostrar se a casa foi reformada ou não

In [ ]:
raw_data = raw_data.assign(renovated=(raw_data['yr_renovated'] > 0).astype(int))

In [ ]:
raw_data.head(10)

In [ ]:
renovated = raw_data.groupby('renovated')['price'].count()
renovated

In [ ]:
renovated_median = raw_data.groupby('renovated')['price'].median()
renovated_median

In [ ]:
plt.figure(figsize=(15, 5))
        
plt.subplot(1, 2, 1)
plt.pie(renovated, explode = (0, 0.1), colors=['r', 'lightblue'], labels= ['Não', 'Sim'], autopct='%1.1f%%')
plt.title('A casa foi renovada ou não?')

plt.subplot(1, 2, 2)
sns.barplot(x=renovated.index, y = renovated_median, palette=['r', 'lightblue'])
plt.title('Mediana dos preços das casas renovadas ou não')
plt.ylabel('Mediana de preço')

plt.tight_layout()
plt.show()

Grande parte das casas não passou por nenhuma renovação, porém, as casas renovadas tendem a possuir um preço mais elevado que as casas não renovadas.

# Respondendo às perguntas

### Quais casas o CEO da *House Rocket* deveria comprar e por qual preço de compra?

De acordo com as análises feitas, o CEO deve considerar adquirir as residências pertencentes ao **grupo 2** (250 a 500 mil dólares) e/ou **grupo 3** (501 mil a 1 milhão de dólares), que tenham sido construidas a partir do **ano de 1991**. Tendo em vista a existência de  alguns fatores, como quantidade de cômodo e área habitacional (que serão melhor abordados na terceira pergunta), que incrementam os preços das residências, é também recomendável a compra de **casas não reformadas**, com intuito de realizar uma reforma.

### Uma vez a casa em posse da empresa, qual o melhor momento para vendê-las e qual seria o preço da venda?

### A *House Rocket* deveria fazer uma reforma para aumentar o preço da venda? Quais seriam as sugestões de mudanças?

As residências com a seguinte descrição tendem a possuir melhores preços de vendas:
* **Quantidade de quartos**: 4
* **Quantidade de banheiros**: 3
* **Área habitacional**: até 60000 pés²
* **Qualidade dos materiais**: 7 ou 8
* **Área acima do solo**: até 60000 pés²
* **Número de andares**: 1 ou 2

Então, se uma residência adquirida possuir uma descrição abaixo dessa, uma reforma com essas mudanças pode aumentar o preço de venda.